<a href="https://colab.research.google.com/github/vshantam/Blindness_Detection/blob/master/Blindness_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install kaggle

In [0]:
!mkdir ~/.kaggle

In [0]:
import json
token = {"username":"vshantam","key":"eb95b0b5d2917ce87b64a974b036c9bd"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
file.close()

In [0]:
!ls -a

.  ..  .config	.kaggle  sample_data


In [0]:
!cp /content/.kaggle/kaggle.json  ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
bradklassen/pga-tour-20102018-data                          PGA Tour Golf Data                                   92MB  2019-08-20 10:32:59           3620  
AnalyzeBoston/crimes-in-boston                              Crimes in Boston                                     10MB  2018-09-04 17:56:03          11587  
matheusfreitag/gas-prices-in-brazil                         Gas Prices in Brazil                                  3MB  2019-07-12 16:23:52           6216  
jealousleopard/goodreadsbooks                               Goodreads-books                                     632KB  2019-06-14 16:35:05          10248  
wkirgsn/electric-motor-temperature                          Elec

In [0]:
!kaggle competitions download -c aptos2019-blindness-detection -p /content

 23% 1.88G/8.01G [00:25<09:53, 11.1MB/s]

In [0]:
!unzip /content/{/content}/competitions/aptos2019-blindness-detection/test_images.zip

In [0]:
!unzip /content/{/content}/competitions/aptos2019-blindness-detection/train_images.zip